# Welcome to Anyscale!

In this Hello World example, we will walk through a few core concepts to get you started with Anyscale. 

Before continuing, please make sure:
* You already have registered an Anyscale account [here](https://console.anyscale.com/).
* Your Anyscale organization has already set up billing or has credit to use.

## Install and import packages

In [ ]:
!pip install ray anyscale art

  Using cached ray-1.9.0-cp38-cp38-macosx_10_15_x86_64.whl (58.7 MB)
  Using cached anyscale-0.4.37-cp38-cp38-macosx_10_9_x86_64.whl
     |████████████████████████████████| 574 kB 3.2 MB/s eta 0:00:01
  Using cached PyYAML-6.0-cp38-cp38-macosx_10_9_x86_64.whl (192 kB)
  Using cached filelock-3.4.0-py3-none-any.whl (9.8 kB)
  Using cached click-8.0.3-py3-none-any.whl (97 kB)
  Using cached redis-4.0.2-py3-none-any.whl (119 kB)
  Using cached msgpack-1.0.3-cp38-cp38-macosx_10_9_x86_64.whl (73 kB)
  Using cached grpcio-1.42.0-cp38-cp38-macosx_10_10_x86_64.whl (4.0 MB)
  Using cached numpy-1.21.4-cp38-cp38-macosx_10_9_x86_64.whl (16.9 MB)
  Using cached protobuf-3.19.1-cp38-cp38-macosx_10_9_x86_64.whl (1.0 MB)
  Using cached sentry_sdk-1.5.0-py2.py3-none-any.whl (140 kB)
  Using cached aioredis-1.3.1-py3-none-any.whl (65 kB)
  Using cached GitPython-3.1.24-py3-none-any.whl (180 kB)
  Using cached pydantic-1.8.2-cp38-cp38-macosx_10_9_x86_64.whl (2.6 MB)
  Using cached requests-2.26.0-py2.py

In [ ]:
import ray
from datetime import datetime
from art import *

## Define a simple function
`@ray.remote` tells Python that the annotated function is to be run on a Ray cluster.

In [ ]:
@ray.remote
def send_random_art(sender):
      return f"Hello, here is a random art from {sender}: {art('rand')}"

## Run the function locally
When you run `ray.init()` without specifying a cluster to connect to, Ray will start a local instance to execute the code. 

In [ ]:
ray.init()
print(ray.get(send_random_art.remote("me")))
ray.shutdown()

## Run the same code on Anyscale


### Set up credential
To authenticate the Ray client to connect to your Anyscale account, copy the command from https://console.anyscale.com/credentials and paste it below to create a file at `~/.anyscale/credentials.json` containing your credentials.

**You only need to do this once on every machine.**


In [ ]:
%%bash

# paste your command below:
mkdir -p ~/.anyscale && echo "{\"cli_token\":\"YOUR_TOKEN\"}" > ~/.anyscale/credentials.json

# Use the next code cell if your Jupyter Notebook is running on a Windows machine

In [ ]:
# %%cmd

# # paste your command below:
# mkdir %HOMEPATH%\.anyscale && echo {"cli_token":"YOUR_TOKEN"} > %HOMEPATH%\.anyscale/credentials.json

### Specify dependency
Here we use [runtime environment](https://docs.ray.io/en/master/dependency-management.html#runtime-environments) to specify the dependencies for executing the code:
* Set the current directory as the working directory so all the files are considered dependencies (although in this example we will exclude every file because this script doesn't have any file dependencies); 
* `datetime` as the PIP package dependency.

In [ ]:
my_env = {
    "working_dir": ".",
    "excludes": ["*"],
    "pip": ["art"]
}

### Run remotely
After running the code block below, it will take a few minutes for Anyscale to start the [cluster](https://docs.anyscale.com/concepts/clusters) named `hello_cluster` and set up the dependencies. You can monitor the progress of the [job](https://docs.anyscale.com/concepts/jobs) either in the cell output, or via the link to the Anyscale Console in the cell output.

In [ ]:
ray.init("anyscale://hello_cluster_" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S"), runtime_env=my_env)
print(ray.get(send_random_art.remote("me")))
ray.shutdown()